## 신경망 학습

- Def
    - 학습
        - 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것
    - 손실함수
        - 학습할 수 있도록 해주는 지표
        - 손실함수의 결괏값을 가장 작게 만드는 가중치 매계변수를 찾는 것이 학습의 목표

- ## 데이터로부터 학습
    - 데이터 주도 학습
        - 데이터로부터 특징(Feature)을 추출하고 그 특징의 패턴을 학습하는 것
        - 다만 적합한 특징을 쓰지 않음녀 좋지 못한 결과

- ## 훈련 데이터와 시험 데이터
    - Objective
        - 범용성을 확인하기 위해
        - 오버피팅 피함
    - train data
    - test data

- ## 손실 함수
    - **신경망 학습의 지표**
        - 해당 지표를 기준으로 최적의 매개변수값 탐색
    - **종류**
        - **오차제곱합**
            - $E = {1 \over 2} \sum (y_k - t_k)^2$
        - **교차 엔트로피 오차**
            - $E = -\sum t_k log y_k$
    - 미니배치학습
        - 훈련 데이터 모두에 대한 손실함수의 합을 구하는 방법
        - $E = -{1\over N}\sum \sum t_{nk}log y_{nk}$
            - 평균손실함수
        - 미니배치
            - 일부 데이터만 꾸려 전체의 근사치로 이용
    - 왜 손실 함수를?
        - 궁극적 목적
            - 높은 정확도를 끌어내는 매개변수
        - 미분
            - 미분값을 단서로 매개변수의 값을 서서히 갱신

In [1]:
def sum_squares_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

- ## 수치 미분
    - 미분
        - 나쁜 구현
            - numerical_diff
                - 해당 구현은 반올림 오차 문제
                    - 작은 값이 생략되어 최종 계산 결과에 오차
                - f 차분
                    - h가 0에 가까워져야 하지만 다가갈 정도는 아님
                    - x + h와 x - h 일때의 차분을 계산
                        - 중심 차분, 중앙 차분
              
           

In [2]:
def numerical_diff(f, x):
    h = 10e-50
    return (f(x + h) - f(x) ) / h
def numerical_diff(f, x):
    h = 1e-4
    return (f(x + h) - f(x - h)) / (2*h)

- ## 기울기
    - 모든 변수의 편미분을 벡터로 정리한 것
    - 기울기가 가르키는 쪽은 각 장소에서 함수의 출력값을 가장 크게 줄이는 방향
- 경사하강법
    - 안정점이 되는 곳에서는 기울기가 0임
    - 경사법은 현 위치에서 기울어진 방향으로 일정 거리만큼 이동하고, 이후 기울기를 다시 구한 후 이동하는 것을 반복하는 것
    - $x_0 = x_0 - \eta {\partial f \over \partial x_0}$
        - $\eta$ : 학습률

In [3]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
    return grad

- ## 신경망에서의 기울기
    - $W = \begin{pmatrix} w_{11} & w_{12} & w_{13} \\ w_{21} & w_{22} & w_{23}\end{pmatrix}$
    - ${\partial W \over \partial W} = \begin{pmatrix} \partial L \over \partial w_{11} & \partial L \over \partial w_{12} & \partial L \over \partial w_{13} \\ \partial L \over \partial w_{21} & \partial L \over \partial w_{22} & \partial L \over \partial w_{23}\end{pmatrix}$    

In [4]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def sigmoid(x):
    return 1 / (1 + np.exp(-x))    


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [5]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3) # 초기화
    def predict(self, x):
        return np.dot(x, self.W)
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

In [6]:
net = simpleNet()
net.W

array([[-0.02155369,  0.80761508, -0.18102304],
       [-0.81391939, -0.11131493, -0.07781729]])

In [7]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
np.argmax(p)

[-0.74545966  0.38438561 -0.17864939]


1

In [8]:
t = np.array([0, 0, 1])
net.loss(x, t)

1.200965771507279

In [9]:
net.W

array([[-0.02155369,  0.80761508, -0.18102304],
       [-0.81391939, -0.11131493, -0.07781729]])

In [10]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x[idx]) # f(x+h)
        x[idx] = tmp_val - h 
        fxh2 = f(x[idx]) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원
        it.iternext()   
    return grad

In [11]:
def f(W):
    return net.loss(x, t)
dW = numerical_gradient(f, net.W)
dW

array([[ 0.10242727,  0.31703057, -0.41945784],
       [ 0.1536409 ,  0.47554586, -0.62918676]])

In [12]:
def function_tmp1(x0):
    return x0 * x0 + 4.0**2.0

In [13]:
numerical_diff(function_tmp1, 3.0)

6.00000000000378

In [14]:
def function_tmp2(x1):
    return 3.0**2.0 + x1*x1


In [15]:
numerical_diff(function_tmp2, 4.0)

7.999999999999119

In [16]:
f = lambda w : net.loss(x, t)
dW = numerical_gradient(f, net.W)
dW

array([[ 0.10242727,  0.31703057, -0.41945784],
       [ 0.1536409 ,  0.47554586, -0.62918676]])

In [17]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)
    def predict(self, x):
        W1, W2 = self.params["W1"], self.params["W2"]
        b1, b2 = self.params["b1"], self.params["b2"]
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y  = softmax(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        t = np.argmax(t, axis = 1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W :self.loss(x,t)
        grad = {}
        grad["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grad["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grad["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grad["b2"] = numerical_gradient(loss_W, self.params["b2"])
        return grad

In [18]:
net = TwoLayerNet(input_size=  784, hidden_size = 100, output_size=  10)
print(net.params["W1"].shape)
print(net.params["b1"].shape)
print(net.params["W2"].shape)
print(net.params["b2"].shape)


(784, 100)
(100,)
(100, 10)
(10,)


In [19]:
x = np.random.rand(100, 784)
y = net.predict(x)

In [27]:
from tensorflow import keras
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784).astype('float32') / 255.0
X_test  = X_test.reshape(10000, 784).astype('float32') / 255.0

In [28]:
X_train.shape

(60000, 784)

In [ ]:
train_loss_list = []
iters_num = 1000
train_size = X_train.shape[0]
batch_size = 100
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
for i in range(iters_num):
    print(i)   
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]
    grad = network.numerical_gradient(X_batch, y_batch)
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(X_batch, y_batch)
    train_loss_list.append(loss)

## 개선

In [35]:
iters_num = 1000
train_size = X_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
for i in range(iters_num):
    print(i)   
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]
    grad = network.numerical_gradient(X_batch, y_batch)
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(X_batch, y_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(X_train, y_train)
        test_acc = network.accuracy(X_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"train acc, test acc : {train_acc, test_acc}")

0


AxisError: axis 1 is out of bounds for array of dimension 1